In [ ]:
import pandas as pd, numpy as np, xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, config_adapter
from helper import RenderJSON

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "10MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
all_dfs = []
file_start_row = pd.DataFrame([{'time (ms)':0, 'family':10, '_T':-1}])
for file in params["dat_files"]:
  event_df = pd.read_csv(file, sep="\t", names=['time (ms)', 'family', 'nbre', '_P', '_V', '_L', '_R', '_T', '_W', '_X', '_Y', '_Z'], skiprows=13, dtype=int)
  all_dfs+=[file_start_row, event_df]
all_dfs = pd.concat(all_dfs, ignore_index=True)
all_dfs

In [ ]:
line_change_df = all_dfs.loc[all_dfs["family"]==10, ['time (ms)', '_T']]
line_change_df

In [ ]:
if (line_change_df["_T"] == line_change_df["_T"].shift(-1, fill_value=pd.NA)).any():
    raise Exception("Filtering issue")

In [ ]:
res={}
for name, node in params["counts"].items():
    res[name] = len(line_change_df.loc[line_change_df["_T"] == node].index)
res = pd.Series(res, name="counts").reset_index()
res

In [ ]:
res.to_csv("counts.tsv", sep="\t", index=False)